### Домашнее задание

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

В рамках конкурса вам нужно предсказать наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра. Датасет сформирован из 100.000 реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:

 

Объективные признаки:

 - Возраст
 - Рост
 - Вес
 - Пол
 

Результаты измерения:

 - Артериальное давление верхнее и нижнее
 - Холестерин
 - Глюкоза
 

Субъективные признаки:

 - Курение
 - Употребление Алкоголя
 - Физическая активность
 

Возраст дан в днях. Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.

Таргет - наличие сердечно-сосудистых заболеваний (ССЗ)

In [3]:
import pandas as pd
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


Разделим наши данные на тренировочную и тестовую выборки

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [10]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Добавим классификатор и запустим кросс-валидацию

In [16]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

classifier1 = Pipeline([
    ('features',feats),
    ('classifier1', LogisticRegression(random_state = 42)),
])

classifier2 = Pipeline([
    ('features',feats),
    ('classifier2', GradientBoostingClassifier(random_state = 42)),
])

classifier3 = Pipeline([
    ('features',feats),
    ('classifier3', RandomForestClassifier(random_state = 42)),
])


#запустим кросс-валидацию
cv_scores1 = cross_val_score(classifier1, X_train, y_train, cv=16, scoring='roc_auc')
cv_score1 = np.mean(cv_scores1)
cv_score_std1 = np.std(cv_scores1)
print('CV score is {}+-{}'.format(cv_score1, cv_score_std1))

#обучим пайплайн на всем тренировочном датасете
classifier1.fit(X_train, y_train)
y_score1 = classifier1.predict_proba(X_test)[:, 1]

#запустим кросс-валидацию
cv_scores2 = cross_val_score(classifier2, X_train, y_train, cv=16, scoring='roc_auc')
cv_score2 = np.mean(cv_scores2)
cv_score_std2 = np.std(cv_scores2)
print('CV score is {}+-{}'.format(cv_score2, cv_score_std2))

#обучим пайплайн на всем тренировочном датасете
classifier2.fit(X_train, y_train)
y_score2 = classifier2.predict_proba(X_test)[:, 1]

#запустим кросс-валидацию
cv_scores3 = cross_val_score(classifier3, X_train, y_train, cv=16, scoring='roc_auc')
cv_score3 = np.mean(cv_scores3)
cv_score_std3 = np.std(cv_scores3)
print('CV score is {}+-{}'.format(cv_score3, cv_score_std3))

#обучим пайплайн на всем тренировочном датасете
classifier3.fit(X_train, y_train)
y_score3 = classifier3.predict_proba(X_test)[:, 1]

CV score is 0.7867401104915408+-0.00852135511666111
CV score is 0.8025125910838183+-0.00707472977074522
CV score is 0.7734501681056019+-0.007171140345435727


Посчитаем precision/recall/f_score

In [19]:
from sklearn.metrics import precision_recall_curve

b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score1)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

precision, recall, thresholds = precision_recall_curve(y_test.values, y_score2)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

precision, recall, thresholds = precision_recall_curve(y_test.values, y_score3)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.386937, F-Score=0.730, Precision=0.647, Recall=0.838
Best Threshold=0.394947, F-Score=0.740, Precision=0.698, Recall=0.788
Best Threshold=0.350000, F-Score=0.719, Precision=0.643, Recall=0.816


### Результат:
Обучены три модели классификации - LogisticRegression, GradientBoostingClassifier и RandomForestClassifier. По приведенным оценкам лучшая модель - градиентный бустинг, что логично, поскольку это самая сложная и долгообучаемая модель из рассмотренных.
    
### Разберем приведенный примеры для сравнения метрик:
   **Первый пример**
    TP - 90
    FP - 10
    FN - 10
    TN - 99890
    TPR = 0.9
    FPR = 0.0001
    Prec = 0.9
    Rec = 0.9


   **Второй пример**
    TP - 90
    FP - 910
    FN - 10
    TN - 98990
    TPR = 0.9
    FPR = 0.009
    Prec = 0.09
    Rec = 0.9
    
Из примеров видно, что в первом случае значения полносты и точности имеют высокие значения, и очевидно что модель работает значительно лучше второй, для которой значение точности резко падает. При этом показатель roc кривой tpr не меняется для обоих моделей, так как количество верно определенных объектов сохраняется. А показатель fpr значительно не изменится, так как объектов класса 0 значительно больше класса 1, и при разумных ошибках модели (то есть не рассматривая случаи когда половину объекто класса 0 модель отнесет к классу 1) эта оценка будет меняться незначительно и не давать полезной информации. 